In [17]:
import numpy as np
from collections import namedtuple
import heapq
import itertools
from random import choice
import time

In [18]:
PUZZLE_DIM = 9
Action = namedtuple('Action', ['pos1', 'pos2'])

# Pre-calculates target positions for heuristics
TARGET_POSITIONS = {
    value: divmod(value-1, PUZZLE_DIM) 
    for value in range(1, PUZZLE_DIM**2)
}

In [19]:
def available_actions(state: np.ndarray)->list[Action]:
    #position of 0
    x,y=[int(_[0]) for _ in np.where(state== 0)]
    actions=[]
    #list of possible directions
    for dx,dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_x, new_y= x + dx, y + dy
        if 0 <= new_x < PUZZLE_DIM and 0 <= new_y < PUZZLE_DIM:
            actions.append(Action((x,y),(new_x,new_y)))
    return actions        

def do_action(state: np.ndarray, act: Action)-> np.ndarray:
    new_state=state.copy()
    new_state[act.pos1], new_state[act.pos2]=new_state[act.pos2],new_state[act.pos1]
    return new_state

In [20]:
def heuristic(state: np.ndarray)->int:
    """ 
  Advanced heuristics combining:
    1. Manhattan distance
    2. Linear conflicts (rows and columns)
    3. Corner tiles detection
    4. Last moves detection
    """
    distance=0
    row_conflicts = 0
    col_conflicts = 0
    #Calculating distance to Manhattan
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value=state[x,y]
            if value != 0:
                target_x,target_y=TARGET_POSITIONS[value]
                distance += abs(x-target_x) + abs(y-target_y)

    # Linear conflicts
    
    # Rows check
    for row in range(PUZZLE_DIM):
        for i in range(PUZZLE_DIM):
            tile_i=state[row,i]
            if tile_i== 0:
                continue
            traget_row_i,_=TARGET_POSITIONS[tile_i]
            # Check if the tile is in the correct row
            if traget_row_i == row:
                for j in range(i+1,PUZZLE_DIM):
                    tile_j=state[row,j]
                    if tile_j != 0 and TARGET_POSITIONS[tile_j][0]== row and tile_i > tile_j:
                        row_conflicts+=2  # Add penalty for linear conflict
    # Column check
    for col in range(PUZZLE_DIM):
        for i in range(PUZZLE_DIM-1):
            tile_i = state[i, col]
            if tile_i == 0:
                continue
            target_row_i, target_col_i = TARGET_POSITIONS[tile_i]
            
            if target_col_i == col:
                for j in range(i + 1, PUZZLE_DIM):
                    tile_j = state[j, col]
                    if tile_j != 0:
                        target_row_j, target_col_j = TARGET_POSITIONS[tile_j]
                        if target_col_j == col and target_row_i > target_row_j:
                            col_conflicts += 2
    # Corner tiles detection
    corner_penalty=0
    corners = [(0,0), (0,PUZZLE_DIM-1), (PUZZLE_DIM-1,0), (PUZZLE_DIM-1,PUZZLE_DIM-1)]
    for x,y in corners:
        value=state[x,y]
        if value !=0:
            target_x,target_y=TARGET_POSITIONS[value]
            if(x,y) != (target_x,target_y) and (target_x,target_y) in corners:
                corner_penalty+=2
    
    # Last moves pattern detection
    last_moves_penalty = 0
    if(PUZZLE_DIM>=4):
        # Checking whether the last two tiles of the final row/column are interchanged
        if state[PUZZLE_DIM-1, PUZZLE_DIM-2] == PUZZLE_DIM**2 - 1 and \
            state[PUZZLE_DIM-1, PUZZLE_DIM-1] == PUZZLE_DIM**2 - 2:
            last_moves_penalty += 6
            
        if state[PUZZLE_DIM-2, PUZZLE_DIM-1] == PUZZLE_DIM**2 - 1 and \
            state[PUZZLE_DIM-1, PUZZLE_DIM-1] == PUZZLE_DIM**2 - 3:
                last_moves_penalty += 6

    return distance+row_conflicts+col_conflicts+corner_penalty+last_moves_penalty



In [ ]:
def a_star(initial_state: np.ndarray):
    open_set = []
    counter = itertools.count()
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    
    # Set for visited states 
    visited = set()
    
    # Initial estimation of heuristics 
    initial_h = heuristic(initial_state)
    heapq.heappush(open_set, (initial_h, next(counter), initial_state, []))
    
    nodes_expanded = 0
    
    while open_set:
        f_score, _, state, path = heapq.heappop(open_set)
        nodes_expanded += 1
        
        if np.array_equal(state, goal_state):
            print(f"Cost: {nodes_expanded}")
            return path,state
        
       
        state_key = str(state.flatten().tolist())
        if state_key in visited:
            continue
            
        visited.add(state_key)
        
        actions = available_actions(state)
        for act in actions:
            new_state = do_action(state, act)
            new_state_key = str(new_state.flatten().tolist())
            
            if new_state_key not in visited:
                g_score = len(path) + 1
                h_score = heuristic(new_state)
                f_score = g_score + h_score
                heapq.heappush(open_set, (f_score, next(counter), new_state, path + [act]))
    
    return None


In [22]:
def generate_puzzle(num_moves=50):
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    
    for _ in range(num_moves):
        acts = available_actions(state)
        state = do_action(state, choice(acts))
    
    return state

In [23]:
initial_state = generate_puzzle(num_moves=160)
print("Iinitial iniziale:")
print(initial_state)

Iinitial iniziale:
[[ 1  2  0 12  5 16  6  8  9]
 [10 11  4 21 14 13 17 18 27]
 [19 20 22  3 23  7 25 35 26]
 [28 29 38 31 15 24 33 43 36]
 [37 39 30 40 32 42 34 45 54]
 [46 47 48 58 41 50 51 44 63]
 [55 56 57 59 49 60 52 61 71]
 [64 65 66 67 68 69 79 53 62]
 [73 74 75 76 77 78 80 70 72]]


In [24]:
start_time = time.time()
solution = a_star(initial_state)
end_time = time.time()
if solution:
    print(f"\nSolution found in {end_time - start_time:.2f} seconds!")
    print("Solution found:", solution[0])
    print("Number of moves(Quality):", len(solution[0]))
    print("Solution:\n",solution[1])
else:
    print("Solution not found.")

Cost: 274068

Solution found in 145.75 seconds!
Solution found: [Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 3)), Action(pos1=(1, 3), pos2=(2, 3)), Action(pos1=(2, 3), pos2=(2, 4)), Action(pos1=(2, 4), pos2=(1, 4)), Action(pos1=(1, 4), pos2=(1, 5)), Action(pos1=(1, 5), pos2=(2, 5)), Action(pos1=(2, 5), pos2=(2, 4)), Action(pos1=(2, 4), pos2=(3, 4)), Action(pos1=(3, 4), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(6, 4)), Action(pos1=(6, 4), pos2=(6, 3)), Action(pos1=(6, 3), pos2=(5, 3)), Action(pos1=(5, 3), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(5, 5)), Action(pos1=(5, 5), pos2=(5, 6)), Action(pos1=(5, 6), pos2=(6, 6)), Action(pos1=(6, 6), pos2=(6, 7)), Action(pos1=(6, 7), pos2=(7, 7)), Action(pos1=(7, 7), pos2=(8, 7)), Action(pos1=(8, 7), pos2=(8, 6)), Action(pos1=(8, 6), pos2=(7, 6)), Action(pos1=(7, 6), pos2=(7, 7)), Action(pos1=(7, 7), pos2=(7, 8)), Action(pos1=(7, 8), pos2=(6, 8)), Action(pos1=(6, 8), pos2=(5, 8)), Action(pos1=(5, 8

This code implements an A* algorithm to solve the puzzle of n boxes. The heuristics used to estimate the distance to the optimal solution include several metrics:

- **Manhattan distance**: calculates the sum of the distances (in terms of vertical and horizontal movements) of each tile from its target position. This is a common and effective measure for sliding puzzles.

- **Linear conflicts**: identifies situations where two or more tiles in the same row or column are in the correct row/column but in the wrong sequence. Adds penalties of 2 units for each conflict detected, improving the accuracy of heuristics.

- **Corner penalty**: checks whether a tile placed in a corner is out of place but its target is also in a corner. This check helps prevent the algorithm from underestimating the cost of complex moves.

- **Penalty for final moves**: detects whether the last two tiles (e.g., those in the final row) are switched relative to their target position, adding a penalty to correct these errors.

This combination of metrics makes the heuristics permissible and more informed, allowing the A* to explore fewer unnecessary nodes and converge more quickly to the solution.